In [28]:
%reload_ext autoreload
%autoreload 2

from inkbloom import EpubReader

book_options = [
    'The Adventures of Sherlock Holmes.epub',
    'Frankenstein.epub',
    'Grimm\'s Fairy Tales.epub',
    'The Hound of the Baskervilles.epub',
    'The Iliad of Homer.epub',
    'A Study in Scarlet.epub',
]

In [29]:
# Read chapters of an epub
ereader = EpubReader(book_options[5])
ereader.read_chapters()
ereader.chapters


/Users/minty/miniconda3/envs/inkbloom/lib/python3.12/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/minty/miniconda3/envs/inkbloom/lib/python3.12/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


['Title: A Study in ScarletAuthor: Arthur Conan DoyleRelease date: April 1, 1995 [eBook #244]\n                Most recently updated: April 17, 2023Language: EnglishCredits: Roger Squires and David Widger',
 '',
 '\n(Being a reprint from the Reminiscences of JOHN H.\nWATSON, M.D., Late of the Army Medical Department.)\n',
 '\nIn the year 1878 I took my degree of Doctor of Medicine of the University of\nLondon, and proceeded to Netley to go through the course prescribed for\nsurgeons in the army. Having completed my studies there, I was duly attached to\nthe Fifth Northumberland Fusiliers as Assistant Surgeon. The regiment was\nstationed in India at the time, and before I could join it, the second Afghan\nwar had broken out. On landing at Bombay, I learned that my corps had advanced\nthrough the passes, and was already deep in the enemy’s country. I followed,\nhowever, with many other officers who were in the same situation as myself, and\nsucceeded in reaching Candahar in safety, where

In [56]:
len(ereader.chapters)

19

In [10]:
ereader.clean_passage(ereader.chapters[3])

'In the year 1878 I took my degree of Doctor of Medicine of the University ofLondon, and proceeded to Netley to go through the course prescribed forsurgeons in the army. Having completed my studies there, I was duly attached tothe Fifth Northumberland Fusiliers as Assistant Surgeon. The regiment wasstationed in India at the time, and before I could join it, the second Afghanwar had broken out. On landing at Bombay, I learned that my corps had advancedthrough the passes, and was already deep in the enemy’s country. I followed,however, with many other officers who were in the same situation as myself, andsucceeded in reaching Candahar in safety, where I found my regiment, and atonce entered upon my new duties.The campaign brought honours and promotion to many, but for me it had nothingbut misfortune and disaster. I was removed from my brigade and attached to theBerkshires, with whom I served at the fatal battle of Maiwand. There I wasstruck on the shoulder by a Jezail bullet, which shatt

In [30]:
chapter = ereader.chapters[3]
input_text = "What are the physical descriptions of characters in the following book chapter? Book chapter: "
query = input_text + chapter

In [41]:
import anthropic
import json

with open('/Users/minty/.secret/anthropic_api_key.json') as f:
    claude_key = json.load(f)["SECRET_KEY"]

client = anthropic.Anthropic(api_key=claude_key)

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system="You are a reader of a book and are tasked with summarizing the physical descriptions of characters and scenes such that someone could paint them based on your descriptions. Return results in a YAML-like format where the dictionary is called Descriptions, each character and scene is prepended with 'Entry: ', and the descriptions have the key 'Descriptions: '. Characters should have 'char -' at the beginning of the description and scenes should have 'scene -' in the beginning of the description. The scene entry should include a short description of what the characters are doing at the scene. If no physical description of a character or scene is provided, do not include it.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": query
                }
            ]
        }
    ]
)


In [42]:
print(message.content[0].text)

Descriptions:
- Entry: Watson
  Description: char - Thin as a lath and brown as a nut, with poor health and a wounded shoulder from war

- Entry: Sherlock Holmes
  Description: char - Has hands mottled with plaster and discolored from strong acids, possesses a grip strength stronger than his appearance would suggest

- Entry: Chemical Laboratory
  Description: scene - A lofty chamber filled with countless bottles, broad low tables scattered about with retorts, test-tubes, and Bunsen lamps with blue flickering flames. Holmes is working alone at a distant table when Watson and Stamford arrive, conducting experiments with blood samples.

Note: While Stamford appears in the text, no physical description is provided for him, so he is not included in the descriptions.


In [43]:
import yaml

# Load the YAML text into a Python dictionary
data = yaml.safe_load(message.content[0].text)

# Convert the structured format to the desired dictionary format
description_dict = {item["Entry"]: item["Description"] for item in data["Descriptions"]}

print(description_dict)

{'Watson': 'char - Thin as a lath and brown as a nut, with poor health and a wounded shoulder from war', 'Sherlock Holmes': 'char - Has hands mottled with plaster and discolored from strong acids, possesses a grip strength stronger than his appearance would suggest', 'Chemical Laboratory': 'scene - A lofty chamber filled with countless bottles, broad low tables scattered about with retorts, test-tubes, and Bunsen lamps with blue flickering flames. Holmes is working alone at a distant table when Watson and Stamford arrive, conducting experiments with blood samples.'}


In [41]:
# Image generation using Dalle3

from openai import OpenAI
import json

with open('/Users/minty/.secret/openai_api_key.json') as f:
    openai_key = json.load(f)["SECRET_KEY"]

client = OpenAI(api_key=openai_key)

prompt= """

Use the following descriptions to generate an image of a scene in an old black and white drawing style.

Descriptions:
- Entry: Watson
  Description: char - Thin as a lath and brown as a nut, with poor health and a wounded shoulder from war

- Entry: Sherlock Holmes
  Description: char - Has hands mottled with plaster and discolored from strong acids, possesses a grip strength stronger than his appearance would suggest

- Entry: Chemical Laboratory
  Description: scene - A lofty chamber filled with countless bottles, broad low tables scattered about with retorts, test-tubes, and Bunsen lamps with blue flickering flames. Holmes is working alone at a distant table when Watson and Stamford arrive, conducting experiments with blood samples.
  """

response = client.images.generate(
  model="dall-e-3",
  prompt=prompt,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

In [42]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-xLnhdzpUnwslS1vjfbWhQ6rc/user-FwQzmM1Ok95YOSfB89jqMuTF/img-sKAFgov5hAHxvPllrLzvSdrq.png?st=2024-11-12T19%3A53%3A59Z&se=2024-11-12T21%3A53%3A59Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-11-12T19%3A55%3A44Z&ske=2024-11-13T19%3A55%3A44Z&sks=b&skv=2024-08-04&sig=X93ZvUadZP/BTcNkN1DnFG6P78MlnRssM9fd7gXuPX4%3D'

# Playing around with Llama 3 finetuning

In [34]:
import json

with open('/Users/minty/.secret/huggingface_token.json') as f:
    hf_token = json.load(f)["TOKEN"]

In [31]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=hf_token,
)
messages = [
    {"role": "system", "content": "You are a reader of a book and are tasked with summarizing the physical descriptions of characters and scenes such that someone could paint them based on your descriptions. Return results in a YAML-like format where the dictionary is called Descriptions, each character and scene is prepended with 'Entry: ', and the descriptions have the key 'Descriptions: '. Characters should have 'char -' at the beginning of the description and scenes should have 'scene -' in the beginning of the description. The scene entry should include a short description of what the characters are doing at the scene. If no physical description of a character or scene is provided, do not include it."},
    {"role": "user", "content": query},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'role': 'assistant', 'content': "Here are the physical descriptions of characters and scenes in the given book chapter:\n\n**Characters:**\n\n- **Sherlock Holmes**: \n  - Entry: 'Sherlock Holmes'\n  - Descriptions: 'char - A brilliant, eccentric, and intelligent man, with a large head, prominent nose, and a pointed chin.'\n  - Scene - 'Holmes' chamber, where he is working at a distant table, surrounded by various chemical equipment and test-tubes.'\n  - Descriptions:'scene - A lofty chamber, lined and littered with countless bottles, broad tables, and Bunsen lamps.'\n\n- **Dr. Watson**: \n  - Entry: 'Dr. Watson'\n  - Descriptions: 'char - A thin, worn man with a pale face and sunken eyes.'\n  - Descriptions:'scene - None, he is simply standing at the Criterion Bar.'\n\n- **Young Stamford**: \n  - Entry: 'Young Stamford'\n  - Descriptions: 'char - A young man with a thin face and brown hair, dressed in formal attire.'\n  - Descriptions:'scene - The Holborn, where he is standing and tal

In [33]:
print(outputs[0]["generated_text"][-1]['content'])

Here are the physical descriptions of characters and scenes in the given book chapter:

**Characters:**

- **Sherlock Holmes**: 
  - Entry: 'Sherlock Holmes'
  - Descriptions: 'char - A brilliant, eccentric, and intelligent man, with a large head, prominent nose, and a pointed chin.'
  - Scene - 'Holmes' chamber, where he is working at a distant table, surrounded by various chemical equipment and test-tubes.'
  - Descriptions:'scene - A lofty chamber, lined and littered with countless bottles, broad tables, and Bunsen lamps.'

- **Dr. Watson**: 
  - Entry: 'Dr. Watson'
  - Descriptions: 'char - A thin, worn man with a pale face and sunken eyes.'
  - Descriptions:'scene - None, he is simply standing at the Criterion Bar.'

- **Young Stamford**: 
  - Entry: 'Young Stamford'
  - Descriptions: 'char - A young man with a thin face and brown hair, dressed in formal attire.'
  - Descriptions:'scene - The Holborn, where he is standing and talking with Watson.'

- **Murray**: 
  - Entry: 'Murra